# 01 - Basic Usage: Poldantic Fundamentals

This notebook demonstrates the core functionality of Poldantic:
- Converting Pydantic models to Polars schemas
- Converting Polars schemas to Pydantic models
- Round-trip conversions

In [1]:
from typing import Optional, List
from pydantic import BaseModel
import polars as pl
from poldantic import to_polars_schema, to_pydantic_model

## 1. Pydantic ➜ Polars

Convert a Pydantic model to a Polars schema.

In [2]:
class User(BaseModel):
    id: int
    name: str
    email: str
    age: int
    active: bool

schema = to_polars_schema(User)
print('Polars Schema:', schema)

Polars Schema: {'id': Int64, 'name': String, 'email': String, 'age': Int64, 'active': Boolean}


In [3]:
# Create DataFrame with schema
data = [
    {'id': 1, 'name': 'Alice', 'email': 'alice@example.com', 'age': 30, 'active': True},
    {'id': 2, 'name': 'Bob', 'email': 'bob@example.com', 'age': 25, 'active': False},
]

df = pl.DataFrame(data, schema=schema)
print(df)

shape: (2, 5)
┌─────┬───────┬───────────────────┬─────┬────────┐
│ id  ┆ name  ┆ email             ┆ age ┆ active │
│ --- ┆ ---   ┆ ---               ┆ --- ┆ ---    │
│ i64 ┆ str   ┆ str               ┆ i64 ┆ bool   │
╞═════╪═══════╪═══════════════════╪═════╪════════╡
│ 1   ┆ Alice ┆ alice@example.com ┆ 30  ┆ true   │
│ 2   ┆ Bob   ┆ bob@example.com   ┆ 25  ┆ false  │
└─────┴───────┴───────────────────┴─────┴────────┘


## 2. Polars ➜ Pydantic

Convert a Polars schema to a Pydantic model.

In [4]:
# Get schema from DataFrame
polars_schema = df.schema

# Convert to Pydantic model
GeneratedUser = to_pydantic_model(polars_schema, 'GeneratedUser', force_optional=False)

# Create instance
user = GeneratedUser(id=3, name='Charlie', email='charlie@example.com', age=28, active=True)
print(user)

id=3 name='Charlie' email='charlie@example.com' age=28 active=True


## 3. Optional Fields

In [5]:
class Product(BaseModel):
    id: int
    name: str
    description: Optional[str] = None
    tags: Optional[List[str]] = None

schema = to_polars_schema(Product)
print('Schema:', schema)

products = [
    {'id': 1, 'name': 'Widget', 'description': 'Useful', 'tags': ['tool']},
    {'id': 2, 'name': 'Gadget', 'description': None, 'tags': None},
]

df = pl.DataFrame(products, schema=schema)
print(df)

Schema: {'id': Int64, 'name': String, 'description': String, 'tags': List(String)}
shape: (2, 4)
┌─────┬────────┬─────────────┬───────────┐
│ id  ┆ name   ┆ description ┆ tags      │
│ --- ┆ ---    ┆ ---         ┆ ---       │
│ i64 ┆ str    ┆ str         ┆ list[str] │
╞═════╪════════╪═════════════╪═══════════╡
│ 1   ┆ Widget ┆ Useful      ┆ ["tool"]  │
│ 2   ┆ Gadget ┆ null        ┆ null      │
└─────┴────────┴─────────────┴───────────┘
